In [ ]:
import os
import gc
import copy
import time
import random
import string

import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import re
from nltk.corpus import stopwords

from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler

In [ ]:
class Config:
    num_classes=1
    epochs=10
    margin=0.5
    model_name = '../input/robertalarge'
    batch_size = 128
    lr = 1e-4
    weight_decay=0.01
    scheduler = 'CosineAnnealingLR'
    max_length = 128
    accumulation_step = 1
    patience = 1

In [ ]:
class ToxicDataset(Dataset):
    def __init__(self, comments, tokenizer, max_length):
        self.comment = comments
        self.tokenizer = tokenizer
        self.max_len = max_length
        
    def __len__(self):
        return len(self.comment)
    
    def __getitem__(self, idx):

        inputs_more_toxic = self.tokenizer.encode_plus(
                                self.comment[idx].lower(),
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )

        
        input_ids = inputs_more_toxic['input_ids']
        attention_mask = inputs_more_toxic['attention_mask']

       
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        }

In [ ]:
class ToxicModel(nn.Module):
    def __init__(self, model_name, args):
        super(ToxicModel, self).__init__()
        self.args = args
        self.model = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(p=0.2)
        self.output = nn.LazyLinear(self.args.num_classes)
    
        
    def forward(self, toxic_ids, toxic_mask):
        
        out = self.model(
            input_ids=toxic_ids,
            attention_mask=toxic_mask,
            output_hidden_states=False
        )
        
        out = self.dropout(out[1])
        outputs = self.output(out)

        return outputs

In [ ]:
def get_predictions(model, dataloader):
    model.eval()
    
    PREDS=[]
    with torch.no_grad():
        bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for step, data in bar:        
            input_ids = data['input_ids'].cuda()
            attention_mask = data['attention_mask'].cuda()

            outputs = model(input_ids, attention_mask)

            PREDS.append(outputs.view(-1).cpu().detach().numpy())

            bar.set_postfix(Stage='Inference')  
        
        PREDS = np.hstack(PREDS)
        gc.collect()

        return PREDS

In [ ]:
df = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')

In [ ]:
args = Config()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [ ]:
def washing_machine(comments):
    corpus=[]
    for i in tqdm(range(len(comments))):
        comment = re.sub('[^a-zA-Z]', ' ', comments[i])
        comment = comment.lower()
        comment = comment.split()
        stemmer = SnowballStemmer('english')
        lemmatizer = WordNetLemmatizer()
        all_stopwords = stopwords.words('english')
        comment = [stemmer.stem(word) for word in comment if not word in set(all_stopwords)]
        comment = [lemmatizer.lemmatize(word) for word in comment]
        comment = ' '.join(comment)
        corpus.append(comment)

    return corpus

In [ ]:
# df=df[:500]

In [ ]:
def validate():
#     comments1 = washing_machine(df['less_toxic'].values)
#     comments2 = washing_machine(df['more_toxic'].values)

    comments1 = df['less_toxic'].values
    comments2 = df['more_toxic'].values

    test_dataset1 = ToxicDataset(comments1, tokenizer, max_length=args.max_length)
    test_loader1 = DataLoader(test_dataset1, batch_size=2*args.batch_size,
                             num_workers=2, shuffle=False, pin_memory=True)
    test_dataset2 = ToxicDataset(comments2, tokenizer, max_length=args.max_length)
    test_loader2 = DataLoader(test_dataset2, batch_size=2*args.batch_size,
                             num_workers=2, shuffle=False, pin_memory=True)

    preds1 = inference(test_loader1)

    preds2 = inference(test_loader2)

    score = []
    for o1,o2 in zip(preds1, preds2):
        s = 1 if o1<o2 else 0
        score.append(s)
    print(np.mean(score))

In [ ]:
def inference(dataloader):
    final_preds = []
    args = Config()
    ### bert-base-uncased
    base_path='../input/rob-large/'
    
    for fold in range(5):
        model = ToxicModel(args.model_name, args)
        model = model.cuda()
        if fold==4 or fold==3:
            base_path='../input/notebookf6da51ad47/'
        
        path = base_path + f'model_fold_{fold}.bin'
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {fold+1}")
        preds = get_predictions(model, dataloader)
        final_preds.append(preds)
    
    
    # bertweet base
    base_path='../input/bertweet/'
    args.model_name = '../input/bertweetbase'
    for fold in range(5):
        model = ToxicModel(args.model_name, args)
        model = model.cuda()
        
        path = base_path + f'model_fold_{fold}.bin'
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {fold+1}")
        preds = get_predictions(model, dataloader)
        final_preds.append(preds)
    
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
# validate()

# Prediction and submission

In [ ]:
sub = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
sub.head(1)

In [ ]:
sub_dataset = ToxicDataset(sub['text'].values, tokenizer, max_length=args.max_length)
sub_loader = DataLoader(sub_dataset, batch_size=2*args.batch_size,
                        num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
sub_preds = inference(sub_loader)

In [ ]:
sub['score'] = sub_preds
sub[['comment_id', 'score']].to_csv('submission.csv', index=False)